In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df_keywords = pd.read_csv('datasets/df_agri_keywords.csv')
df_ner = pd.read_csv('datasets/df_agri_ner.csv')
df_sentiment = pd.read_csv('datasets/processed_results.csv')

In [ ]:
# Add columns to df_final
ner_cols = ['actor_persons', 'actor_organizations', 'actor_locations']
sentiment_cols = ['sentiment', 'sentiment_confidence'] 
df_keywords[ner_cols] = df_ner[ner_cols]
df_keywords[sentiment_cols] = df_sentiment[sentiment_cols]
df_final = df_keywords
df_final.to_csv('datasets/df_agri_final.csv', index=False, encoding='utf-8')

In [14]:
df_final.head()

,channel_title,channel_name,country,segment_id,segment_start,segment_end,duration_seconds,num_keywords,avg_density,themes,...,secondary_categories,text_type,keywords,keywords_filtered,nouns_in_keywords,actor_persons,actor_organizations,actor_locations,sentiment,sentiment_confidence
0,France 2,france2,france,5788,2025-12-03 06:34:04.031 +0100,2025-12-03 06:36:02.025 +0100,118,18,13.4,", attenuation_climatique_solutions_indirectes,...",...,['energy'],segment,"['voyez', 'radiologues', 'entraîne activement ...","['entraîne activement lucie robert', 'stations...","['concurrence', 'stations', 'résultats', 'dire...",['mathieu'],[],"['france', 'doubs']",positive,0.859769
1,M6,m6,france,3445,2025-11-17 15:20:24.048 +0100,2025-11-17 15:21:26.064 +0100,62,4,5.3,", biodiversite_causes_indirectes, changement_c...",...,[],segment,"['enseigne travaille', 'va voir si', 'david', ...","['enseigne travaille', 'cours', 'prochaine car...","['temps', 'directeurs', 'retraites', 'site', '...","['david', 'christo popov', 'thomas fersen']",[],['paris'],positive,0.851858
2,M6,m6,france,1379,2025-05-13 20:14:11.059 +0200,2025-05-13 20:15:50.004 +0200,99,8,8.1,"attenuation_climatique_solutions_indirectes, b...",...,[],segment,"['réglementations strictes imposées', 'essaie'...","['réglementations strictes imposées', 'parc na...","['heure', 'mérite', 'sympathie', 'euros', 'rég...","['saint-pierre', 'rouget rascasse rouge']",[],"['îles des caraïbes', 'parc national de port-c...",negative,0.928592
3,TF1,tf1,france,4852,2025-12-25 09:11:59.056 +0100,2025-12-25 09:13:11.082 +0100,72,15,17.2,"adaptation_climatique_solutions, attenuation_c...",...,[],segment,"['allez pouvoir', 'froid', 'herbes', 'myrtille...","['certains papiers', 'plantes', 'déchets', 'hu...","['année', 'pommes', 'andalou', 'choses', 'limi...",['noël sinon'],[],[],negative,0.488315
4,France 2,france2,france,5744,2025-11-30 07:36:06.090 +0100,2025-11-30 07:38:37.058 +0100,151,11,10.5,"biodiversite_causes, biodiversite_concepts_gen...",...,[],segment,"['voyez', 'côté alpes', 'météo', 'peu gêné', '...","['côté alpes', 'météo', 'extérieur', 'choses',...","['météo', 'choses', 'extérieur', 'règles', 'en...","['sébastien thomas regardez', 'cana vérandas',...",[],"['alsace', 'etats-unis', 'pyrénées', ""l' illin...",positive,0.851335
